In [ ]:
from efficient_probit_regression.datasets import Covertype
from efficient_probit_regression.sampling import leverage_score_sampling, truncated_normal
from scipy.stats import multivariate_normal, norm, truncnorm

import plotly.express as px

import numpy as np
import pandas as pd

from tqdm import tqdm

In [ ]:
def gibbs_sampler_probit(
    X: np.ndarray,
    y: np.ndarray,
    prior_mean: np.ndarray,
    prior_cov: np.ndarray,
    num_samples,
    num_chains,
    burn_in=100,
    probabilities=None,
    original_size=None
):
    n, d = X.shape

    if probabilities is None:
        probabilities = np.full(n, 1 / n)

    factor_squared = 1 / (probabilities * n)

    prior_cov_inv = np.linalg.inv(prior_cov)
    B = np.linalg.inv(prior_cov_inv + X.T @ np.multiply(X, factor_squared[:, np.newaxis]))

    beta_start = np.zeros(d)  # TODO: set this to the MLE

    def simulate_chain():
        beta = beta_start
        samples = []
        for i in tqdm(range(num_samples + burn_in)):
            a = np.where(y == -1, -np.inf, 0)
            b = np.where(y == -1, 0, np.inf)

            # sample latent variables
            latent_mean = X @ beta
            # latent_mean = np.ones(n)
            # latent_mean = np.zeros(n)
            latent = truncated_normal(
                a,
                b,
                mean=latent_mean,
                std=1,
                size=n,
            )
            # latent = np.zeros(n)
            # latent = 2 * np.ones(n)

            beta_mean = B @ (prior_cov_inv @ prior_mean + X.T @ (latent * factor_squared))
            beta = multivariate_normal.rvs(size=1, mean=beta_mean, cov=B)

            samples.append(beta)

        return np.array(samples[burn_in:])

    if num_chains == 1:
        samples = simulate_chain()
    # else:
    #     sample_chunks = Parallel(n_jobs=num_chains)(
    #         delayed(simulate_chain)() for i in range(num_chains)
    #     )
    #     samples = np.vstack(sample_chunks)

    return samples

In [ ]:
dataset = Covertype()

prior_mean = np.zeros(dataset.get_d())
prior_cov = 10 * np.eye(dataset.get_d())

In [ ]:
num_samples_gibbs = 1000
sample_original = gibbs_sampler_probit(
    X=dataset.get_X(),
    y=dataset.get_y(),
    prior_mean=prior_mean,
    prior_cov=prior_cov,
    num_samples=num_samples_gibbs,
    num_chains=1, 
    probabilities=None, 
    original_size=dataset.get_n()
)

In [ ]:
sample_size = 500
X_reduced, y_reduced, weights = leverage_score_sampling(
    X = dataset.get_X(),
    y = dataset.get_y(), 
    sample_size = sample_size, 
    augmented = True, 
    online = False, 
    round_up = True
)

probabilities = 1 / (weights * sample_size)
sample_reduced = gibbs_sampler_probit(
    X=X_reduced,
    y=y_reduced,
    prior_mean=prior_mean,
    prior_cov=prior_cov,
    num_samples=num_samples_gibbs,
    num_chains=1, 
    probabilities=probabilities, 
    original_size=dataset.get_n()
)

In [ ]:
df_original = pd.DataFrame(sample_original)
df_original["method"] = "original"
df_original["sample_index"] = list(range(num_samples_gibbs))

df_reduced = pd.DataFrame(sample_reduced)
df_reduced["method"] = "reduced"
df_reduced["sample_index"] = list(range(num_samples_gibbs))

df = pd.concat([df_original, df_reduced], ignore_index=True)

beta_index = 0
fig = px.line(df, x = "sample_index", y=beta_index, color="method", title=f"beta_{beta_index}")
fig.show()

In [ ]:
index = 0

print(np.var(df_original[index]))
print(np.var(df_reduced[index]))

print(np.mean(df_original[index]))
print(np.mean(df_reduced[index]))

In [ ]:
df_melted = df.melt(id_vars=["method"])
df_melted.head()

fig = px.box(df_melted, x="variable", y="value", color="method")
fig.show()